# Day5_2: 실습 프로젝트 - 더러운 전자상거래 데이터 정제

## 📚 프로젝트 목표

**Part 1: 데이터 품질 진단**
1. 결측치 탐색 및 처리 전략 수립하기
2. 이상치 탐지 및 비즈니스 로직 검증하기
3. 데이터 타입 오류 파악하기

**Part 2: 데이터 정제 실행**
1. 결측치 처리 (삭제/대체)
2. 이상치 처리 및 수정
3. 데이터 타입 변환 및 포맷 통일
4. 중복 레코드 제거
5. Plotly 대시보드로 전체 과정 시각화
6. 정제된 데이터 저장

---

## 🎯 프로젝트 개요

| 항목 | 내용 |
|------|------|
| **데이터셋** | 전자상거래 주문 데이터 (510건) |
| **주요 문제** | 결측치, 이상치, 타입 오류, 중복, 비즈니스 로직 위배 |
| **목표** | 분석 가능한 깨끗한 데이터 생성 |
| **결과물** | 정제된 CSV + Plotly 대시보드(HTML) |

**분석가 관점**: 실제 업무 데이터는 항상 "더럽습니다". 데이터 정제는 전체 분석 시간의 70%를 차지하는 핵심 작업입니다!

---

## 📦 환경 설정

### 필요한 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("✅ 라이브러리 임포트 완료")

### 데이터 로드

In [ ]:
# 더러운 데이터 로드
df = pd.read_csv('datasets/dirty_ecommerce_data.csv')

print(f"📊 원본 데이터 로드 완료: {len(df)}건")
print(f"📋 컬럼 수: {len(df.columns)}개")
df.head()

---

# Part 1: 데이터 품질 진단

---

## Mission 1: 전체 데이터 구조 파악하기

### 📋 미션 설명
- 데이터의 기본 정보 확인
- 각 컬럼의 데이터 타입 확인
- 메모리 사용량 파악

In [ ]:
# TODO: df.info()로 기본 정보 확인


In [ ]:
# TODO: df.dtypes로 각 컬럼의 타입 확인


---

## Mission 2: 결측치 탐색하기

### 📋 미션 설명
- 각 컬럼별 결측치 개수 파악
- 결측치 비율 계산
- Plotly 막대 차트로 시각화

In [ ]:
# TODO: df.isnull().sum()으로 컬럼별 결측치 개수 확인


In [ ]:
# TODO: 결측치 비율(%) 계산
# 힌트: (결측치 개수 / 전체 레코드 수) * 100


In [ ]:
# TODO: Plotly 막대 차트로 결측치 시각화
# 힌트: px.bar() 사용


### 💡 결측치 처리 전략 수립

| 컬럼 | 처리 방법 | 이유 |
|------|----------|------|
| customer_grade | 최빈값(mode) 대체 | 등급은 범주형, '일반'이 가장 많을 것 |
| delivery_status | '준비중'으로 대체 | 상태 누락은 초기 단계로 추정 |
| payment_method | '카드'로 대체 | 가장 일반적인 결제 수단 |
| region | 최빈값 대체 | 서울/경기가 가장 많을 것 |
| price | 해당 제품 평균가로 대체 | 같은 제품은 비슷한 가격 |

**원칙**: 비즈니스 도메인 지식을 활용하여 합리적인 대체값 선택!

---

## Mission 3: 이상치 탐지하기

### 📋 미션 설명
- 수량(quantity) 분포 확인 (음수 값 있는지)
- 가격(price) 분포 확인 (너무 높거나 낮은 값)
- 박스플롯으로 시각화

In [ ]:
# TODO: quantity 기술 통계 확인
# 힌트: df['quantity'].describe()


In [ ]:
# TODO: 음수 수량 레코드 찾기
# 힌트: df[df['quantity'] < 0]


In [ ]:
# TODO: price 기술 통계 확인 (타입 오류 제외 - 숫자만)
# 힌트: pd.to_numeric(df['price'], errors='coerce') 사용


In [ ]:
# TODO: Plotly 박스플롯으로 price 분포 시각화
# 힌트: px.box()


### 💡 이상치 판단 기준

| 이상치 | 판단 기준 | 처리 방법 |
|--------|----------|----------|
| 음수 수량 | quantity < 0 | 절대값으로 변환 (반품으로 추정) |
| 비정상 고가 | price > 백만원 | 제품 평균가로 대체 |
| 비정상 저가 | price < 100원 | 제품 평균가로 대체 |

**원칙**: 삭제보다는 수정! 데이터 손실 최소화

---

## Mission 4: 데이터 타입 오류 찾기

### 📋 미션 설명
- order_date 컬럼의 다양한 날짜 형식 확인
- category, delivery_status의 공백 문제 확인
- customer_grade의 대소문자 혼용 확인
- price 컬럼의 문자열 혼입('원') 확인

In [ ]:
# TODO: order_date의 고유값 샘플 확인
# 힌트: df['order_date'].sample(20)


In [ ]:
# TODO: category 고유값에서 공백 문제 확인
# 힌트: df['category'].unique()


In [ ]:
# TODO: customer_grade의 대소문자 문제 확인
# 힌트: df['customer_grade'].value_counts()


In [ ]:
# TODO: price에서 '원' 텍스트 포함된 레코드 찾기
# 힌트: df[df['price'].astype(str).str.contains('원', na=False)]


---

## Mission 5: 중복 레코드 확인하기

### 📋 미션 설명
- 완전 중복 레코드 찾기
- order_id 중복 확인

In [ ]:
# TODO: 완전 중복 레코드 개수 확인
# 힌트: df.duplicated().sum()


In [ ]:
# TODO: order_id 중복 확인
# 힌트: df['order_id'].duplicated().sum()


---

## Mission 6: 비즈니스 로직 검증하기

### 📋 미션 설명
- 취소/반품 주문인데 total_amount > 0인 경우 찾기
- total_amount가 quantity * price와 일치하지 않는 경우 찾기

In [ ]:
# TODO: 취소/반품인데 금액이 있는 레코드 찾기
# 힌트: df[(df['delivery_status'].isin(['취소', '반품'])) & (df['total_amount'] > 0)]


### 💡 비즈니스 로직 검증 규칙

| 규칙 | 조건 | 처리 방법 |
|------|------|----------|
| 취소/반품 금액 | delivery_status가 '취소' 또는 '반품' | total_amount를 0으로 수정 |
| 금액 계산 오류 | total_amount ≠ quantity × price | 재계산하여 수정 |

**원칙**: 비즈니스 규칙 위반은 반드시 수정!

---

# Part 2: 데이터 정제 실행

---

## Mission 7: 데이터 타입 정제하기

### 📋 미션 설명
1. order_date를 datetime 타입으로 변환 (다양한 형식 처리)
2. category, delivery_status의 공백 제거
3. customer_grade 대소문자 통일 (대문자로)
4. price에서 '원' 텍스트 제거 후 숫자로 변환

In [ ]:
# TODO: order_date를 datetime으로 변환
# 힌트: 먼저 '/'와 공백 제거 후 pd.to_datetime() 사용


In [ ]:
# TODO: category와 delivery_status 공백 제거
# 힌트: str.strip() 사용


In [ ]:
# TODO: customer_grade 대문자로 통일
# 힌트: str.upper() 사용


In [ ]:
# TODO: price에서 '원' 제거 후 숫자로 변환
# 힌트: str.replace('원', '') 후 pd.to_numeric()


---

## Mission 8: 이상치 처리하기

### 📋 미션 설명
1. 음수 수량을 양수로 변환
2. 비정상적인 가격(100원 미만 또는 100만원 이상)을 제품별 평균가로 대체

In [ ]:
# TODO: 음수 수량을 양수로 변환
# 힌트: df.loc[df['quantity'] < 0, 'quantity'] = df['quantity'].abs()


In [ ]:
# TODO: 비정상 가격을 제품별 평균가로 대체
# 힌트: groupby('product')['price'].transform('mean') 사용


---

## Mission 9: 결측치 처리하기

### 📋 미션 설명
- customer_grade: 최빈값으로 대체
- delivery_status: '준비중'으로 대체
- payment_method: '카드'로 대체
- region: 최빈값으로 대체
- price: 제품별 평균가로 대체

In [ ]:
# TODO: customer_grade 최빈값으로 대체
# 힌트: fillna(df['customer_grade'].mode()[0])


In [ ]:
# TODO: delivery_status를 '준비중'으로 대체


In [ ]:
# TODO: payment_method를 '카드'로 대체


In [ ]:
# TODO: region 최빈값으로 대체


In [ ]:
# TODO: price를 제품별 평균가로 대체
# 힌트: groupby + transform


---

## Mission 10: 중복 레코드 제거하기

### 📋 미션 설명
- 완전 중복 레코드 제거
- 첫 번째 레코드만 유지

In [ ]:
# TODO: 중복 레코드 제거
# 힌트: drop_duplicates() 사용


---

## Mission 11: 비즈니스 로직 수정하기

### 📋 미션 설명
1. 취소/반품 주문의 total_amount를 0으로 수정
2. 모든 레코드의 total_amount 재계산

In [ ]:
# TODO: 취소/반품 주문의 total_amount를 0으로


In [ ]:
# TODO: total_amount 재계산
# 힌트: df['total_amount'] = df['quantity'] * df['price']


---

## Mission 12: Plotly 대시보드 생성하기

### 📋 미션 설명
정제 전후 비교 대시보드를 생성합니다:
1. 결측치 비교 (정제 전/후)
2. 레코드 수 변화
3. 가격 분포 변화
4. 카테고리별 매출 현황

In [ ]:
# TODO: 2x2 서브플롯 생성
# 힌트: make_subplots(rows=2, cols=2) 사용


---

## Mission 13: 정제된 데이터 저장하기

### 📋 미션 설명
- CSV 파일로 저장
- 최종 품질 리포트 출력

In [ ]:
# TODO: 정제된 데이터 CSV로 저장
# 힌트: to_csv('datasets/clean_ecommerce_data.csv', index=False, encoding='utf-8-sig')


In [ ]:
# TODO: 최종 품질 리포트 출력
print("===== 데이터 정제 완료 =====")
print(f"원본 레코드 수: {원본_레코드수}")
print(f"정제 후 레코드 수: {len(df)}")
print(f"제거된 레코드: {원본_레코드수 - len(df)}")
print(f"\n남은 결측치: {df.isnull().sum().sum()}")
print(f"\n정제된 파일: datasets/clean_ecommerce_data.csv")

---

## 📊 프로젝트 완료 체크리스트

### Part 1: 진단 단계
- [ ] Mission 1: 데이터 구조 파악
- [ ] Mission 2: 결측치 탐색 및 시각화
- [ ] Mission 3: 이상치 탐지 및 박스플롯
- [ ] Mission 4: 데이터 타입 오류 파악
- [ ] Mission 5: 중복 레코드 확인
- [ ] Mission 6: 비즈니스 로직 검증

### Part 2: 정제 단계
- [ ] Mission 7: 데이터 타입 변환
- [ ] Mission 8: 이상치 처리
- [ ] Mission 9: 결측치 대체
- [ ] Mission 10: 중복 제거
- [ ] Mission 11: 비즈니스 로직 수정
- [ ] Mission 12: Plotly 대시보드 생성
- [ ] Mission 13: 정제 데이터 저장

---

## 💡 학습 정리

### 데이터 정제 핵심 프로세스

| 단계 | 작업 | 핵심 메소드 |
|------|------|------------|
| 1. 진단 | 결측치/이상치/타입오류 파악 | info(), isnull(), describe() |
| 2. 타입 변환 | 날짜/문자열/숫자 포맷 통일 | pd.to_datetime(), str.strip() |
| 3. 이상치 처리 | 극단값 수정 또는 대체 | loc[], transform() |
| 4. 결측치 대체 | 최빈값/평균/중앙값 대체 | fillna(), mode(), groupby |
| 5. 중복 제거 | 완전 중복 삭제 | drop_duplicates() |
| 6. 검증 | 비즈니스 규칙 재확인 | 조건 필터링 |
| 7. 저장 | 정제 데이터 파일 출력 | to_csv() |

### 💡 실무 팁

1. **항상 원본 보존**: df_clean = df.copy()로 작업
2. **단계별 검증**: 각 정제 작업 후 결과 확인
3. **비즈니스 우선**: 도메인 지식 활용한 합리적 대체
4. **시각화 필수**: Plotly로 정제 전후 비교
5. **문서화 습관**: 정제 규칙을 주석으로 기록

**데이터 정제는 지루하지만 가장 중요한 작업입니다!**